In [65]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
import os
os.chdir("/content/drive/My Drive/mercari_capstone2")
!ls -l

total 1713858
-rw------- 1 root root    2441360 Jun 23 10:05  1-EDA.ipynb
-rw------- 1 root root      40868 Jun 25 11:18 '2-featureization and pickleing.ipynb'
-rw------- 1 root root      46260 Jun 25 15:25  3_Hyperparameter_tuning.ipynb
-rw------- 1 root root      44151 Jun 25 15:26 '4-train deploy.ipynb'
-rw------- 1 root root     122880 Jun 24 19:02  Mercari_to1.db
drwx------ 2 root root       4096 Jun 23 11:42  pickle
-rw------- 1 root root   11853920 Jun 25 11:18  price_log.pickle
-rw------- 1 root root 1402624694 Jun 25 11:18  tfidf.pickle
-rw------- 1 root root  337809843 Nov 11  2017  train.tsv


In [67]:
!pip uninstall scikit-learn -y
!pip install scikit-learn==1.0.2

Found existing installation: scikit-learn 1.0.2
Uninstalling scikit-learn-1.0.2:
  Successfully uninstalled scikit-learn-1.0.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached scikit_learn-1.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.5 MB)


In [68]:
#importing modules/libraries
import pandas as pd
import numpy as np
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import os
import psutil
# from scipy.stats import randint as sp_randint
# from scipy.stats import uniform as sp_uniform


from tqdm.notebook import tqdm
# from collections import Counter
# from collections import defaultdict
import re as regex
import random
# from random import sample
# from bs4 import BeautifulSoup
import pickle
import inspect
import time

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelBinarizer,OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.linear_model import Lasso,Ridge


import string
# import emoji
# from wordcloud import WordCloud
import nltk
nltk.download("stopwords")
# nltk.download("brown")
# nltk.download("names")
# nltk.download('punkt')
nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('universal_tagset')
# from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
# from nltk.stem.porter import PorterStemmer





import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [69]:
def Datapoint():

  data_dict = {}

  name = input('Your product name\n')



  item_condition_id  = int(input('Condition of your product ranging from 1 to 5 from good to worse \n'))


  category_name  = input('What all Categories your product belongs to separeted by a "/" \n')


  brand_name  = input('What brand is your product \n')


  shipping	  = int(input('0 for shipping charges/ 1 for no shipping charges\n'))


  item_description  = input('describe your product succintly \n')

  if name == '':
    name ='abs'
  if brand_name == '':
    brand_name ='abs'
  if item_description == '':
    item_description ='No Description Yet'

  data_dict['name'] = name.strip()
  data_dict['item_condition_id'] = item_condition_id
  data_dict['category_name'] = category_name.strip()
  data_dict['brand_name'] = brand_name.strip()
  data_dict['shipping'] = shipping
  data_dict['item_description'] = item_description.strip()

  df = pd.DataFrame.from_records([data_dict])

  df['item_condition_id'] = df['item_condition_id'].astype('category')
  df['shipping'] = df['shipping'].astype('category')
  df['name'] = df['name'].astype('object')
  df['category_name'] = df['category_name'].astype('object')
  df['brand_name'] = df['brand_name'].astype('object')
  df['item_description'] = df['item_description'].astype('object')

  return df

In [70]:
def Impute(df,clock):
  # imputing with a 'abs' or absent
    # df['category_name'].fillna(value='abs', inplace=True)
    df['name'].fillna(value='abs', inplace=True)
    df['item_description'].fillna(value='No Description Yet', inplace=True)

    # using brands from train data only


  # imputing with a 'abs' or absent for train data and using  brand
  # names form train data only as target and freqency encoding done
  # in later sections ofnotebook so avoiding data leakage
    with open('pickle/impute_brand_name','rb') as f:
      brand_name = pickle.load(f)
    df.loc[~df['brand_name'].isin(brand_name),'brand_name'] = 'abs'

    with open('pickle/impute_category_name','rb') as f:
      category_name = pickle.load(f)
    df.loc[~df['category_name'].isin(category_name),'category_name'] = 'abs'

   # print fucntion completion time and with function name
    print(f'[{round((time.time() - clock),2)}] {inspect.stack()[0][3]} completed')
    del brand_name,category_name
    return df

In [71]:
# a category column contains Nan or 3 or more sub category in it upto 5
# as rows with more than 3 or less than 3 categories are less than 0.1 percent,
# we make only 3 new cols with segregated category names
def sub_cat(row):
      try:
        split = row.split('/')
        if len(split) >= 3:
          return split[0],split[1],split[2]
        if len(split) == 2:
          return split[0], split[1], 'abs'
        elif len(split) == 1:
          return split[0], 'abs', 'abs'
        else:
          return 'abs', 'abs', 'abs'
      except Exception:
          return  'abs', 'abs', 'abs'

# extracting extra features from data
def Extract_features(df,clock):
  # regex used later in this section to count number of them in a text column
    RE_PUNCTUATION = '|'.join([regex.escape(x) for x in string.punctuation])
    non_alphanumpunct = regex.compile(u'[^A-Za-z0-9\.?!,; \(\)\[\]\'\"\$]+')

  # extracting sub categories
    print(f'[{round(time.time()-clock)}]Extracting Subcat')
    df['sc1'], df['sc2'],df['sc3'] =  zip(*df['category_name'].apply(sub_cat))
    df.drop(columns='category_name',inplace = True)
    df['sc1'] = df['sc1'].astype('category')
    df['sc2'] = df['sc2'].astype('category')
    df['sc3'] = df['sc3'].astype('category')

  # has description or not/ missing value added as a feature
    print(f'[{round(time.time()-clock)}]Extracting HasDescription ')
    df['HasDescription'] = 1
    df.loc[df['item_description']=='No description yet', 'HasDescription'] = 0
    df['HasDescription'] =df['HasDescription'].astype('category')

  # has price or not/ [rm] values in textual columns are indicative of presence
  # price in the data which has been cleaned as suggested by the compition itself
    print(f'[{round(time.time()-clock)}]Extracting HasPrice ')
    df['HasPrice'] = 0
    df.loc[df['item_description'].str.contains('[rm]', regex=False), 'HasPrice'] = 1
    df.loc[df['name'].str.contains('[rm]', regex=False), 'HasPrice'] = 1
    df['HasPrice'] =df['HasPrice'].astype('category')

  # counting number of tokens in textual columns
    print(f'[{round(time.time()-clock)}]Extracting Token Count ')
    df['NameTokenCount'] = df['name'].str.split().apply(len)
    df['DescTokenCount'] = df['item_description'].str.split().apply(len)
    df['NameTokenCount'] = df['NameTokenCount'].astype('uint32')
    df['DescTokenCount'] = df['DescTokenCount'].astype('uint32')

  # ratio of token token counts in name and description columns(2 textual cols)

    print(f'[{round(time.time()-clock)}]Extracting Name to Desc token Ratio ')
    df['NameDescTokenRatio'] = df['NameTokenCount']/df['DescTokenCount']
    df['NameDescTokenRatio'] =df['NameDescTokenRatio'].astype('float32')

  # adding missing value as a feature for brand
    print(f'[{round(time.time()-clock)}]Extracting HasBrand ')
    df['HasBrand'] =1
    df.loc[df['brand_name'] == 'abs', 'HasBrand'] = 0
    df['HasBrand'] =df['HasBrand'].astype('category')

  # counting uppper and lower count of characters as EDA suggested phoney/
  # counterfiet items when listed uses too many bold and Caps charactes with emojis
    print(f'[{round(time.time()-clock)}]Extracting Lower count ')
    df['NameLowerCount'] = df.name.str.count('[a-z]')
    df['DescriptionLowerCount'] = df.item_description.str.count('[a-z]')
    df['NameLowerCount'] =df['NameLowerCount'].astype('uint32')
    df['DescriptionLowerCount'] =df['DescriptionLowerCount'].astype('uint32')


    print(f'[{round(time.time()-clock)}]Extracting Upper count ')
    df['NameUpperCount'] = df.name.str.count('[A-Z]')
    df['DescriptionUpperCount'] = df.item_description.str.count('[A-Z]')
    df['NameUpperCount'] =df['NameUpperCount'].astype('uint32')
    df['DescriptionUpperCount'] =df['DescriptionUpperCount'].astype('uint32')

  # punctuation count
    print(f'[{round(time.time()-clock)}]Extracting Punctuation Count ')
    df['NamePunctCount'] = df.name.str.count(RE_PUNCTUATION)
    df['DescriptionPunctCount'] = df.item_description.str.count(RE_PUNCTUATION)
    df['NamePunctCount'] = df['NamePunctCount'].astype('uint32')
    df['DescriptionPunctCount'] = df['DescriptionPunctCount'].astype('uint32')

  # punct count ratio
    print(f'[{round(time.time()-clock)}]Extracting Punctuation Ratio ')
    df['NamePunctCountRatio'] = df['NamePunctCount'] / df['NameTokenCount']
    df['DescriptionPunctCountRatio'] = df['DescriptionPunctCount'] / df['DescTokenCount']
    df['NamePunctCountRatio'] =df['NamePunctCountRatio'].astype('float32')
    df['DescriptionPunctCountRatio'] =df['DescriptionPunctCountRatio'].astype('float32')

  # digit count( if model can get a sense of bundled items)
    print(f'[{round(time.time()-clock)}]Extracting Digit count ')
    df['NameDigitCount'] = df.name.str.count('[0-9]')
    df['DescriptionDigitCount'] = df.item_description.str.count('[0-9]')
    df['NameDigitCount'] =df['NameDigitCount'].astype('uint32')
    df['DescriptionDigitCount'] =df['DescriptionDigitCount'].astype('uint32')

  # emoji and/or other nonalphanum count
    print(f'[{round(time.time()-clock)}]Extracting NonAlphaNum count ')
    df['NonAlphaDescCount'] = df['item_description'].str.count(non_alphanumpunct)
    df['NonAlphaNameCount'] = df['name'].str.count(non_alphanumpunct)
    df['NonAlphaDescCount'] = df['NonAlphaDescCount'].astype('uint32')
    df['NonAlphaNameCount'] = df['NonAlphaNameCount'].astype('uint32')


    cols = set(df.columns.values)
    non_num_col = {'name', 'item_condition_id', 'brand_name',
                  'shipping', 'item_description', 'sc1',
                  'sc2', 'sc3','HasDescription','HasPrice','HasBrand'
                  }

    cols_to_normalize = cols - non_num_col

  # normalizing all the counts and ratios
    print(f'[{round(time.time()-clock)}]Normalizing')
    df_to_normalize = df[list(cols_to_normalize)]
    with open('pickle/normalize_min','rb') as f:
      min = pickle.load(f)
    with open('pickle/normalize_max','rb') as f:
      max = pickle.load(f)
    df_to_normalize = (df_to_normalize - min) / (max - min)


    df = df[list(non_num_col)]
    df = pd.concat([df, df_to_normalize],axis=1)

    # df.drop(columns='index',inplace=True)

    del(df_to_normalize)


    '''  extracting mean categorical and brand price with minding data leakage with addition of
    random noise so making data more robust. An idea taken form a some youtube video of a
    kaggle grandmaster . This noise addition clearly has impacted the performace of model very positively. '''

    print(f'[{round(time.time()-clock)}]Extracting Mean price Categories')
    mean_dc = {}

    lst = ['sc1','sc2','sc3', 'brand_name']

  #imputing values for nan with mean
    def boundary_case(hmap,key):
      try:
        return float(hmap[key])*np.random.normal(1,0.1)
      except:
        '''  when cases in test data are not
        present in train data mean_dict[feat] returns a nan to tackle that this part
        has been added( tho with normal usage it does not occur as this has been
        taken care of in  the imputation part itself , i had an experiment run which
        produced those cases so made this part as permanent only) '''
        np.mean(list(hmap.values()))*np.random.normal(1,0.1)


    for feat in lst:
        ''' for every categorical column in the list above  finding the mean price of
        every category in it and adding that price in a column with a noise added to it
        *np.randon.normal(1,0.1)'''
        with open(f'pickle/mean_dc_{feat}','rb') as f:
          hmap = pickle.load(f)

        df['MeanPrice_'+feat] = df[feat].apply(lambda x : boundary_case(hmap,str(x))).astype(np.float32)
        df['MeanPrice_'+feat].fillna(np.mean(list(hmap.values()))*np.random.normal(1,0.1), inplace=True  )



    print(f'[{round((time.time() - clock),2)}] {inspect.stack()[0][3]} completed')

    return df

In [72]:
def Make_text_column(df,clock):
    '''As we saw in EDA that brands with NAN values can be imputed with names and item_desciption
    columns as there are brand names prevelent with more than 40 to 45 percent of chance.
    So instead of imputing so many brands (43 percent), just creating a new column by merging
    brands_name, name and item_description and making a text column and letting tfidf taking care of it.
    '''

    df['text'] = df['name'].astype(str)+' '+df['brand_name'].str.strip().astype(str)+' '+df['item_description'].str.strip().astype(str)
    df = df.drop(columns=['item_description'])


    def decontracted(text):
    # tried many kinds of regex to clean the data but final result wasnt effected much wiht
    # this part so only doing necessary onces
        try:
            text = regex.sub(u"won't", "will not", text)
            text = regex.sub(u"can\'t", "can not", text)
            text = regex.sub(u"n\'t", " not", text)
            text = regex.sub(u"\'t", " not", text)
            # separating digits for a sense of count if bundled items sold
            text = u" ".join(regex.split('(\d+)',text) )

        except:
            print('error')
        return text

    def clean(df,col):
        non_alphanums = regex.compile(u'[^A-Za-z0-9 ]+')
        # wl = WordNetLemmatizer()

        preprocessed_text = []
        for _,sentance in tqdm(df[col].iteritems(),total=df.shape[0]):

            sentance = decontracted(sentance)

            # nonalphanumeric character removal
            sentance = non_alphanums.sub(u' ', sentance)

            ''' did not lemmatize cause takes a lot of time and has negligible to no
            effect on performance
            did not convert to lower case as this takes a lot of time and can be done
            interensicly within TFIDF and Count vectorization along with text standardization'''
            # lemmetizing
            # sentance = ' '.join(wl.lemmatize(word.strip()) for word in sentance.split())
            sentance = ' '.join(word.strip() for word in sentance.split())

            preprocessed_text.append(sentance)
        df[col] = pd.Series(preprocessed_text).values
        del preprocessed_text
        return df


    print('Cleaning text')
    df= clean(df,'text')
    print(f'Done')

    print(f'[{round((time.time() - clock),2)}] {inspect.stack()[0][3]} completed')


    return df

In [73]:
def high_categorical(df,col='brand_name'):
        ''' As brand has close to 5000 categories, converting it to numbers
        by Frequency encoding brand with minding data leakage, Tho  lgbm handles high order
        categorical columns efficiently this encoding boosted performance instead of just
        using OHE, also it is with addition of random noise so making data more robust and
        enhancing performance I had chosen to do the same with sub categorical columns but the
        did not perform that well and took extra memory space too'''
        with open('pickle/brand_freq_dict','rb') as f:
          dict_replace = pickle.load(f)
        col_cat: pd.Series.astype('float16') = df[col].map(dict_replace)

        return col_cat.values.reshape(-1,1)

'''Converting all the data till yet to numeric form if not yet done'''
def Convert_to_predictor(df,clock,stopwords=stopwords,high_categorical=high_categorical):
    try:
      df.drop(columns='index',inplace = True)
    except:
      pass

    with open('pickle/item_condition_id','rb') as f:
      cv = pickle.load(f)
    item_condition_id = cv.transform(df['item_condition_id'].astype(int).iloc[:].values.reshape(-1,1))
    print(f'[{round((time.time() - clock),2)}]Transform item_condition_id  data completed.{item_condition_id.shape}')


    with open('pickle/shipping','rb') as f:
      cv = pickle.load(f)
    shipping = cv.transform(df['shipping'].astype(int).iloc[:].values.reshape(-1,1))
    print(f'[{round((time.time() - clock),2)}]Transform shipping  data completed.{shipping.shape}')


    with open('pickle/HasDescription','rb') as f:
      cv = pickle.load(f)
    HasDescription = cv.transform(df['HasDescription'].astype(int).iloc[:].values.reshape(-1,1))
    print(f'[{round((time.time() - clock),2)}]Transform HasDescription  data completed.{HasDescription.shape}')


    with open('pickle/HasPrice','rb') as f:
      cv = pickle.load(f)
    HasPrice = cv.transform(df['HasPrice'].astype(int).iloc[:].values.reshape(-1,1))
    print(f'[{round((time.time() - clock),2)}]Transform HasPrice  data completed.{HasPrice.shape}')


    with open('pickle/HasBrand','rb') as f:
      cv = pickle.load(f)
    HasBrand = cv.transform(df['HasBrand'].astype(int).iloc[:].values.reshape(-1,1))

    print(f'[{round((time.time() - clock),2)}]Transform HasBrand  data completed.{HasBrand.shape}')

    with open('pickle/sc1','rb') as f:
      cv = pickle.load(f)
    sc1 = cv.transform(df.sc1.astype(str).iloc[:].values.reshape(-1,1))
    print(f'[{round((time.time() - clock),2)}]Transform sc1  data completed.{sc1.shape}')

    with open('pickle/sc2','rb') as f:
      cv = pickle.load(f)
    sc2 = cv.transform(df.sc2.astype(str).iloc[:].values.reshape(-1,1))
    print(f'[{round((time.time() - clock),2)}]Transform sc2  data completed.{sc2.shape}')

    with open('pickle/sc3','rb') as f:
      cv = pickle.load(f)
    sc3 = cv.transform(df.sc3.astype(str).iloc[:].values.reshape(-1,1))
    print(f'[{round((time.time() - clock),2)}]Transform sc3  data completed.{sc3.shape}')




    df_dummies = scipy.sparse.hstack([item_condition_id, shipping, HasDescription, HasPrice,HasBrand,sc1,sc2,sc3])
    print(df_dummies.shape)
    df.drop(columns=['item_condition_id', 'shipping','HasDescription', 'HasPrice','HasBrand'],inplace=True)
    df.drop(columns=['sc1','sc2','sc3'],inplace=True)

    print(f'[{round((time.time() - clock),2)}]Transform categories data completed.')


    cols = ['NamePunctCount', 'NameDigitCount', 'DescriptionDigitCount',\
            'NameUpperCount', 'DescriptionPunctCount', 'DescriptionPunctCountRatio', \
            'DescTokenCount', 'DescriptionUpperCount', 'NonAlphaDescCount', \
            'NonAlphaNameCount', 'NameTokenCount', 'NameLowerCount', \
            'NameDescTokenRatio', 'DescriptionLowerCount', 'NamePunctCountRatio',\
            'MeanPrice_sc1','MeanPrice_sc2','MeanPrice_sc3','MeanPrice_brand_name']

    df_num = scipy.sparse.csc_matrix(df[cols].values)
    print(df_num.shape)

    df.drop(columns=cols,inplace=True)

    print(f'[{round((time.time() - clock),2)}]Transform numeric  data completed.')

    with open('pickle/count_vect','rb') as f:
          vect = pickle.load(f)

    df_name = scipy.sparse.csr_matrix(vect.transform(df.name))

    print(f'[{round((time.time() - clock),2)}]Transform name data completed.')



    with open('pickle/tfidf_vect','rb') as f:
          vect = pickle.load(f)

    df_text = scipy.sparse.csr_matrix(vect.transform(df.text))


    print(f'[{round((time.time() - clock),2)}]Transform text data completed.')




    # frequency encoding brands
    df_brand = high_categorical(df)
    df_brand= scipy.sparse.csr_matrix(df_brand)

    print(f'[{round((time.time() - clock),2)}]Transform brand data completed.')



    df_merge = scipy.sparse.hstack((df_brand,df_dummies, df_num,df_name, df_text ))
    print('Merge all data completed.')


    del df_brand,df_dummies,df_num,df_text,df_name
    print(f'[{round((time.time() - clock),2)}] {inspect.stack()[0][3]} complete')

    return df_merge

In [74]:
clock = time.time()
# df,y = Data(clock,n_rows =-1)
df = Datapoint()
df = Impute(df,clock)
df = Extract_features(df,clock)
df = Make_text_column(df,clock)
df = Convert_to_predictor(df,clock)

KeyboardInterrupt: ignored

In [75]:
sklearn.__version__

'1.0.2'

In [76]:
def error(y_test, predictions):
  return np.sqrt(mean_squared_error( y_test, predictions ))

In [77]:
df.shape

(1, 151022)

In [78]:
np.isnan(df.data).sum()

0

In [79]:
!pwd
!ls

/content/drive/MyDrive/mercari_capstone2
 1-EDA.ipynb				 pickle
'2-featureization and pickleing.ipynb'	 price_log.pickle
 3_Hyperparameter_tuning.ipynb		 tfidf.pickle
'4-train deploy.ipynb'			 train.tsv
 Mercari_to1.db


In [82]:
with open('pickle/models/ridge_weights','rb') as f:
  rdg = pickle.load(f)
op_rdg = rdg.predict(df.tocsr())

In [83]:
with open('pickle/models/lbgm_coeffs','rb') as f:
  lgbm = pickle.load(f)
op_lgbm = lgbm.predict(df.tocsr())

In [91]:
le = 0.4395201992375062
re = 0.4443864614855584
tot = le+re

In [92]:
new_price = op_lgbm*re/tot + op_rdg*le/tot

In [93]:
pred = {'ridge_price':op_rdg, 'lgbm_price':op_lgbm, 'weighted raito price' :new_price}

In [94]:
pred

{'ridge_price': array([57.6634912]),
 'lgbm_price': array([3.49072175]),
 'weighted raito price': array([30.57710647])}